In [51]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Preprocessing

In [52]:
df = pd.read_csv('/content/drive/MyDrive/FWE 458/Lec7/Wcr_GPPdaily.csv')

# rename columns
df.rename(columns={'TA_F': 'TA', 'SW_IN_F': 'SW', 'VPD_F': 'VPD', 'GPP_NT_VUT_REF': 'GPP'}, inplace=True)

# drop TIMESTAMP column
df.drop(columns=['TIMESTAMP'], errors='ignore', inplace=True)
# check if missing values
df.dropna(inplace=True)
df.head()

,TA,SW,VPD,GPP
0,-20.063,72.603,0.413,-0.517364
1,-12.814,12.358,0.147,-0.094241
2,-12.625,33.132,0.128,-0.166819
3,-18.652,93.481,0.263,-0.582301
4,-20.269,45.502,0.261,-0.568240


# Feature Engineering

In [53]:
# create interaction terms
df['SW_VPD'] = df['SW'] * df['VPD']
df['SW_TA'] = df['SW'] * df['TA']
df['VPD_TA'] = df['VPD'] * df['TA']
df.head()

,TA,SW,VPD,GPP,SW_VPD,SW_TA,VPD_TA
0,-20.063,72.603,0.413,-0.517364,29.985039,-1456.633989,-8.286019
1,-12.814,12.358,0.147,-0.094241,1.816626,-158.355412,-1.883658
2,-12.625,33.132,0.128,-0.166819,4.240896,-418.291500,-1.616000
3,-18.652,93.481,0.263,-0.582301,24.585503,-1743.607612,-4.905476
4,-20.269,45.502,0.261,-0.568240,11.876022,-922.280038,-5.290209


# Linear Regression using scipy optimization

In [54]:
# create the linear model using the provided equation
def linear_model(params, SW, VPD, TA):
    β0, β1, β2, β3, β4, β5, β6 = params
    return β0 + β1 * SW + β2 * VPD + β3 * TA + β4 * (SW * VPD) + β5 * (SW * TA) + β6 * (VPD * TA)

# define the cost function using mse
def cost_function(params, SW, VPD, TA, GPP):
    GPP_pred = linear_model(params, SW, VPD, TA)
    mse = np.mean((GPP_pred - GPP) ** 2)
    return mse
# extract features
SW = df['SW']
VPD = df['VPD']
TA = df['TA']
GPP = df['GPP']

# initial guess for parameters
initial_guess = np.zeros(7)

# perform optimization
result = minimize(cost_function, initial_guess, args=(SW, VPD, TA, GPP))

# extract optimized parameters
β0, β1, β2, β3, β4, β5, β6 = result.x

# print coefficients
print("Coefficients from scipy:", "\n")
print("β0:", β0)
print("β1:", β1)
print("β2:", β2)
print("β3:", β3)
print("β4:", β4)
print("β5:", β5)
print("β6:", β6, "\n")
# print r square
GPP_pred = linear_model(result.x, SW, VPD, TA)
print("R-squared:", r2_score(GPP, GPP_pred))

Coefficients from scipy: 

β0: -944.6391385761914
β1: -3.245829012213404
β2: -648.3947259737711
β3: 82.15079008795271
β4: 1.7916242530440003
β5: -0.45600827057720106
β6: 13.597198669184277 

R-squared: 0.01358269810244761


# Linear regression using sklearn regression function

In [55]:
# define features
x = df[['SW', 'VPD', 'TA', 'SW_VPD', 'SW_TA', 'VPD_TA']]
y = df['GPP']
# fit model
model = LinearRegression()
model.fit(x, y)
# print coefficients
print("Coefficients from sklearn:", "\n", "\n",
      "β0:", model.intercept_, "\n",
      "β1:", model.coef_[0], "\n",
      "β2:", model.coef_[1], "\n",
      "β3:", model.coef_[2], "\n",
      "β4:", model.coef_[3], "\n",
      "β5:", model.coef_[4], "\n",
      "β6:", model.coef_[5], "\n",
      )
# print r square
R2 = model.score(x, y)
print("R-squared:", R2)

Coefficients from sklearn: 
 
 β0: -944.0672018203164 
 β1: -3.249149679810575 
 β2: -648.5741732975123 
 β3: 82.16998132930851 
 β4: 1.7924915107911943 
 β5: -0.45605035694939033 
 β6: 13.595402450063377 

R-squared: 0.013582700933254976


# Compare models

From the above results, we can see that the R-sqaure are the same for both models. And the coefficients obtained by both models are also the same. That
makes same because we are fitting the same data using the same model, just with different implementing methods. There might have some slight difference between the values if looking closer into the decimal places. That could due to the inherent randomness of the model itself.